# Packmol Interface

Packmol is a tool for creating initial configurations for molecular dynamics simulations by packing molecules into a given region of space. molify provides a Python interface to Packmol, making it easy to build molecular systems. 

You need a working Packmol installation for this feature. For more about Packmol, see https://github.com/m3g/Packmol

In [ ]:
from rdkit.Chem import Draw

import molify

## Basic Usage: Creating a Molecular Box

The `pack()` function creates periodic boxes filled with molecules at a specified density.

### Simple Example: Water Box

In [ ]:
# Generate water conformers
water = molify.smiles2conformers("O", numConfs=10)

print(f"Generated {len(water)} water conformers")
print(f"Each conformer: {water[0]}")

In [ ]:
# Pack 10 water molecules into a box
water_box = molify.pack(
    data=[water],  # List of conformer lists
    counts=[10],  # Number of each molecule type
    density=1000,  # Density in kg/m³
)

print(f"Water box: {water_box}")
print(f"Cell dimensions: {water_box.cell.lengths()} Å")
print(f"Total atoms: {len(water_box)}")
print(f"Volume: {water_box.get_volume():.2f} Å³")

The ``connectivity`` information from the input ``Atoms`` object is preserved in the output box.

In [ ]:
# Connectivity is preserved!
print(f"Has connectivity: {'connectivity' in water_box.info}")
print(f"Number of bonds: {len(water_box.info['connectivity'])}")
print(f"Expected bonds: {10 * 2} (10 molecules x 2 O-H bonds each)")

## Parameters Explained

### `data`: List of Conformer Lists

`data` is a list where each element is a list of conformers (ASE Atoms objects) for one molecule type.

```python
data = [
    [water_conf1, water_conf2, ...],    # Water conformers
    [ethanol_conf1, ethanol_conf2, ...] # Ethanol conformers
]
```

If you need more molecules than conformers, they will be duplicated:
- 10 conformers + counts=[100] → conformers are reused

### `counts`: Number of Each Molecule

```python
counts=[10, 5]  # 10 of first type, 5 of second type
```

### `density`: Target Density (kg/m³)

The box size is automatically calculated to match this density.

### `packmol`: Executable Path (Optional)

By default, `pack()` automatically detects your Packmol installation. It searches for one of the following:
- **Native binary** (`packmol`)
- **Julia version** (`packmol.jl`)

You can explicitly specify which version to use:

```python
# Let auto-detection choose (recommended)
molify.pack([water], [10], 1000)

# Explicitly use Julia version
molify.pack([water], [10], 1000, packmol="packmol.jl")

# Use custom path
molify.pack([water], [10], 1000, packmol="/path/to/packmol")
```

To check which executable will be used:
```python
executable = molify.find_packmol_executable()
print(f"Using: {executable}")
```

In [ ]:
# Check which executable will be used
executable = molify.find_packmol_executable()
print(f"Auto-detection found: {executable}")

## Creating Mixtures

### Water + Ethanol Mixture

In [ ]:
# Generate conformers for both molecule types
water = molify.smiles2conformers("O", numConfs=10)
ethanol = molify.smiles2conformers("CCO", numConfs=10)

print(f"Water conformers: {len(water)}")
print(f"Ethanol conformers: {len(ethanol)}")

In [ ]:
# Create mixture: 5 water + 5 ethanol
mixture = molify.pack(
    data=[water, ethanol],
    counts=[5, 5],
    density=800,  # kg/m³
)

print(f"Mixture: {mixture}")
print(f"Cell: {mixture.cell.lengths()} Å")
print(f"Total atoms: {len(mixture)}")

### Visualizing the Mixture

In [ ]:
# Convert to RDKit for visualization
mixture_mol = molify.ase2rdkit(mixture)

# Visualize
Draw.MolToImage(mixture_mol, size=(600, 400))

## Working with Packed Systems

### Analyzing the Box

In [ ]:
# Calculate actual density
from molify.utils import calculate_density

actual_density = calculate_density(mixture)
print(f"Density: {actual_density:.1f} kg/m³")

## Key Takeaways

### What We Learned

1. **`pack()` is simple**: ASE → ASE transformation
   - Only changes atomic positions
   - Preserves all connectivity information
   - Preserves bond orders, charges, etc.

2. **Parameters**:
   - `data`: List of conformer lists (one per molecule type)
   - `counts`: How many of each molecule
   - `density`: Target density in kg/m³
   - `packmol`: Optional - auto-detects if not specified

3. **Auto-detection**: Automatically finds Packmol installation
   - Prefers Julia version (`packmol.jl`) for better compatibility
   - Falls back to native binary in PATH
   - Can be overridden with explicit path

4. **Conformers are reused** if counts > available conformers